pip install:

Python packages used in this notebook:
- pandas
- os
- langchain-openai
- dotenv
- langchain-community
- langchain
- langchain-core
- 

AZURE

In [ ]:
import os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = "3293d148178a4cf39f5b3026cb621eb1" 
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://fish-health-bsc-resource-aiservices1116524949.openai.azure.com/"

model = AzureChatOpenAI(
     openai_api_version="2023-07-01-preview",
     azure_deployment="gpt-4",
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-07-01-preview",
)

OPENAI


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(
    model="gpt-3.5-turbo"
)



GET AL ENVIROMENT VARIABLES

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

TAVILY - Built in search engine tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

search.invoke("Hvordan er været i Geilo, Norge?")

[{'url': 'https://www.timeanddate.no/vaer/norge/geilo/siste-uke',
  'content': 'Været som var i Geilo for siste 24 timer Vis været for: Siste 24 timer 19. mars 2024 18. mars 2024 17. mars 2024 16. mars 2024 15. mars 2024 14. mars 2024 13. mars 2024 12. mars 2024 11. mars 2024 10. mars 2024 9. mars 2024 8. mars 2024 7. mars 2024 6. mars 2024 5. mars 2024 4. mars 2024'},
 {'url': 'https://www.yr.no/nb/værvarsel/timetabell/5-25630/Norge/Buskerud/Hol/Geilo?i=2',
  'content': '0 til 0.4. 0-0,4. 3 m/s fra sørvest med vindkast på 9 m/s. 3 (9) Svak vind fra sørvest med vindkast på 9 m/s. Sol opp 06:21. Sol ned 18:47. Forrige dag Neste dag. Dagens værvarsel for Geilo time for time.'},
 {'url': 'https://www.timeanddate.no/vaer/norge/geilo',
  'content': 'Været i Geilo, Norge. Været. Tidssone. Sommertid. Sol og måne. Været i dag Time for time 14-dagers varsel I går/været som var Klima (gjennomsnitt) Nå.'},
 {'url': 'https://www.yr.no/nb/værvarsel/daglig-tabell/1-112132/Norge/Buskerud/Hol/Geilo',


Adding a vectorestore with the Langsmith documentation

In [8]:
tools = [search]

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [18]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at e

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith, developed by LangChain, is a platform specifically designed to help with building production-grade LLM applications. One of its features includes providing support for testing. This platform allows you to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. Seamless integration with LangChain, the popular open-source framework for building with LLMs, is also provided. \n\nTo get started with testing in LangSmith, you can refer to the tracing and evaluation quick start guides provided in the platform's documentation. Other sections in the user guide also explain the workflows that LangSmith supports at each stage of the LLM application lifecycle.\n\nPlease note that before you begin, you'll need to create an account, obtain an API key, and configure your environment."}

In [12]:
agent_executor.invoke({"input": "Hvordan er været i Åros i Røyken, Norge?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in Åros, Røyken, Norway'}`


[{'url': 'https://www.wunderground.com/forecast/no/åros', 'content': 'Current Weather for Popular Cities . San Francisco, CA warning 48 ... Åros, Viken, Norway 10-Day Weather Forecast star_ratehome. 17 ...'}, {'url': 'https://weather.com/weather/tenday/l/Åros+Viken+Norway?canonicalCityId=89fab4fd4a4171e85b636661f200f9395bee4a1777531f2fe70171f4e2585a80', 'content': 'Be prepared with the most accurate 10-day forecast for Åros, Viken, Norway with highs, lows, chance of precipitation from The Weather Channel and Weather.com'}, {'url': 'https://www.yr.no/en/forecast/daily-table/1-59711/Norway/Akershus/Asker/Åros', 'content': 'Saturday 9 Mar. 5° / -1°. 0 mm. 2 m/s. Open hourly forecast. Updated 22:36.How often is the weather forecast updated? Forecast as PDF Forecast as SVG. Weather forecast for Åros for the next 10 days.'}, {'url': 'https://www.timean

{'input': 'Hvordan er været i Åros i Røyken, Norge?',
 'output': 'Været i Åros, Røyken, Norge varierer sterkt, men for øyeblikket er det rundt 5° / -1°. Det er forventet å være noe overskyet. For mer detaljert informasjon om været i Åros, kan du besøke nettsteder som The Weather Channel, Wunderground eller yr.no.'}

ADDING IN MEMORY

In [20]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

In [22]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you further?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you further?'}

KEEP TRACK OF MESSAGES AUTOMATICALLY

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [24]:
message_history = ChatMessageHistory()

In [25]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}